In [1]:
import numpy as np
import pandas as pd
import pulp

# The Transshipment Problem

This example is a more general version of the Transportation Problem, which failed to consider that there can be multiple ways to get from one point to another.

<img src="03 - The Transshipment Problem image 01.png">

In this example, we will imagine that the potato chip factory Sørlandschips AS has started up three facilities for producing their products `(A, D, J)`. These factories can send bags of potato chips to three different customers `(E, G, H)`. Similarly to the Transportation Problem, the factories have a maximum production capacity and the customers have a limited demand. But in this case, there are a lot of differnt route alternatives between each pair of factory and customer. In this example, we want to ensure that all customer demands are met (total of 2700 units) at a minimum cost, where the production capacity is actually larger than the demand (total of 3000 units).

## Prepare the data

In [2]:
Nodes = ['A','B','C','D','E','F','G','H','I','J']
Transshipment_nodes = ['B', 'C', 'F', 'I']
Supply_nodes = ['A', 'D', 'J']
Customer_nodes = ['E', 'G', 'H']

### Shipping cost

In [3]:
# Shipping cost
data = [
    [np.nan,     59, np.nan,     86, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
    [    59, np.nan,    318, np.nan,    215, np.nan, np.nan,    112, np.nan, np.nan],
    [np.nan,    318, np.nan,    219,    137, np.nan, np.nan, np.nan, np.nan, np.nan],
    [    86, np.nan,    219, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
    [np.nan,    215,    137, np.nan, np.nan, np.nan, np.nan,    160,    117, np.nan],
    [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan,    107,     94, np.nan,     63],
    [np.nan, np.nan, np.nan, np.nan, np.nan,    107, np.nan,     98, np.nan, np.nan],
    [np.nan,    112, np.nan, np.nan,    160,     94,     98, np.nan,    132, np.nan],
    [np.nan, np.nan, np.nan, np.nan,    117, np.nan, np.nan,    132, np.nan,    128],
    [np.nan, np.nan, np.nan, np.nan, np.nan,     63, np.nan, np.nan,    128, np.nan]
]

cost = pd.DataFrame(data=data, index=Nodes, columns=Nodes)
cost.fillna(' ')

,A,B,C,D,E,F,G,H,I,J
A,,59,,86,,,,,,
B,59,,318,,215,,,112,,
C,,318,,219,137,,,,,
D,86,,219,,,,,,,
E,,215,137,,,,,160,117,
F,,,,,,,107,94,,63
G,,,,,,107,,98,,
H,,112,,,160,94,98,,132,
I,,,,,117,,,132,,128
J,,,,,,63,,,128,


### Production capacity

In [4]:
data = {'A': 1200, 'D': 1100, 'J': 700}
capacity = pd.Series(data, name='Production capacity')
capacity

A    1200
D    1100
J     700
Name: Production capacity, dtype: int64

### Customer demand

In [5]:
data = {'E': 900, 'G': 1200, 'H': 600}
demand = pd.Series(data, name='Customer demand')
demand

E     900
G    1200
H     600
Name: Customer demand, dtype: int64

## Create the variables

We define the variable $x_{i,j}$ to be the number of units to be sent from node $i$ to node $j$. Since each nodes is not connected directly to all the other nodes, we only need to create variables for the connections that actually exsist. If you count, you will see that there are 15 different routes on the map. Further, since each route between any two nodes should support sending units in both directions, we need two variables (one for each direction). This mean that we end up with a total of 30 variables to describe this problem.

In [6]:
# Let us first extract only the values the correspond to a route between to nodes and save them as a dictionary
routes = {k1: {k2:v2 for k2,v2 in v1.items() if pd.notnull(v2)} for k1, v1 in cost.to_dict().items()}
routes

{'A': {'B': 59.0, 'D': 86.0},
 'B': {'A': 59.0, 'C': 318.0, 'E': 215.0, 'H': 112.0},
 'C': {'B': 318.0, 'D': 219.0, 'E': 137.0},
 'D': {'A': 86.0, 'C': 219.0},
 'E': {'B': 215.0, 'C': 137.0, 'H': 160.0, 'I': 117.0},
 'F': {'G': 107.0, 'H': 94.0, 'J': 63.0},
 'G': {'F': 107.0, 'H': 98.0},
 'H': {'B': 112.0, 'E': 160.0, 'F': 94.0, 'G': 98.0, 'I': 132.0},
 'I': {'E': 117.0, 'H': 132.0, 'J': 128.0},
 'J': {'F': 63.0, 'I': 128.0}}

In [7]:
x = pulp.LpVariable.dicts('route',
                          ((i, j) for i in routes for j in routes[i]),
                          lowBound = 0,
                          cat='Integer')

## Initiate an empty LP Problem

In [8]:
prob = pulp.LpProblem("TransshipmentProblem", pulp.LpMinimize)

## Create the constraints

### Transshipment nodes: What goes in, must come out

For the transshipment nodes `(B, C, F, I)`, there is no room for storing units. This means that every unit that is sent in to any of these nodes must also leave it. One way to formulate this is to say the the sum of all units entering this node must equal the sum of all units leaving this node. To take a specific example, look at node `B`. It should be able to receive units from all nodes that it is directly connected to, meaning the pairs `((A,B), (C,B), (E,B), (H,B))`. Similarly, it should also be possible to send units to these neighbouring nodes, meaning the pairs `((B,A),(B,C),(B,E),(B,H))`. Since we have defined each direction as a separate variable, we can enforce this behaviour for node `B` as follows:
$$x_{B,A} + x_{B,C} + x_{B,E} + x_{B,H} = x_{A,B} + x_{C,B} + x_{E,B} + x_{H,B}$$

or simply

$$\textrm{units out} =\textrm{units in}$$

In [9]:
for i in Transshipment_nodes:
    node_out = pulp.lpSum([x[i,j] for j in routes[i]])
    node_in = pulp.lpSum([x[j,i] for j in routes[i]])
    
    prob += node_out == node_in, f"Transhipment-node {i}"

### Supply nodes (source)

A source node is a node that has a capacity to generate units. In this case, the source nodes correspond to the potato chip factories located in node `(A, D, J)`. One could easily make the mistake of assuming that the constraint for these nodes would be that they can only deliver out the same ampunt of units that they can produce. However, if e.g. factory `D` want to send units to node `B` _through_ node `A`, this will not be possible under the suggested constraint, as the number of units exiting node `A`would the nbe greater than the capacity it is able to produce by itself. Instead, we must limit the outgoing stream of units to the sum of the source nodes capacity AND the all other units _entering_ the node. Finally, since we want to allow the factory to send out less units than it has the capacity to produce, we insert this as a $\leq$-constraint. In the example above, where units should be allowed to pass through node `A`, the constraint will look like this:
$$x_{A,B} + x_{A,D} \leq 1200 + x_{D,A} + x_{B,A}$$

or simply

$$\textrm{units out} \leq \textrm{capacity} + \textrm{units in}$$

In [10]:
for i in Supply_nodes:
    node_out = pulp.lpSum([x[i,j] for j in routes[i]])
    node_in = pulp.lpSum([x[j,i] for j in routes[i]])
    
    prob += node_out <= capacity[i] + node_in, f"Supply-node {i}"

### Customer nodes (sink)

Analogous to the previous constraint, the customers in the destination nodes should be able to "absorb" units, meaning that more units should be allowed to enter these nodes than units exiting them. Remember from the problem statement that we want to ensure that the demand is met. This means that we will use an equality constraint rather than the inequality constraint in the previous  subchapter. For example, at customer node `G`, it should be allowed to keep 1200 of the units it has received.
$$x_{G,H} + x_{G,F} = x_{H,G} + x_{F,G} + 1200$$

or simply

$$\textrm{units in} = \textrm{units out} + \textrm{demand}$$

In [11]:
for i in Customer_nodes:
    node_out = pulp.lpSum([x[i,j] for j in routes[i]])
    node_in = pulp.lpSum([x[j,i] for j in routes[i]])
    
    prob += node_in == node_out + demand[i], f"Customer-node {i}"

## Create the objective function
If any scenario $x_{i,j}$ is carried out, it will bring with it a cost in the form of the shipping fee to transfer that many units from point $i$ to point $j$. The objective function is therefor to minimize the sum of units sent multiplied by the shipping cost per unit.

In [12]:
prob += pulp.lpSum([cost.loc[i, j] * x[i, j] for i in routes for j in routes[i]])

## Find the optimal solution

In [13]:
prob.solve()
status = pulp.LpStatus[prob.status]
obj_value = prob.objective.value()

print(f"The solver found a solution that is *{status}*, where the total cost spent on shipping is {obj_value:,.2f} NOK")

The solver found a solution that is *Optimal*, where the total cost spent on shipping is 668,300.00 NOK


In [14]:
results = pd.DataFrame.from_dict(data={i: {j: x[i,j].value() for j in routes[i]} for i in routes}, orient='index', dtype=int)
results.replace(0, np.nan).fillna('')

,B,D,A,C,E,H,I,G,J,F
A,1200,,,,,,,,,
B,,,,,100,1100,,,,
C,,,,,800,,,,,
D,,,,800,,,,,,
E,,,,,,,,,,
F,,,,,,,,700,,
G,,,,,,,,,,
H,,,,,,,,500,,
I,,,,,,,,,,
J,,,,,,,,,,700
